# Snowpylot demo for Western Snow Conference 2025
Demonstrating the functionality of the snowpylot library using snowpits from the 2020-2024 water years


In [12]:
# Import libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
from snowpylot.caaml_parser import caaml_parser

### Set Up

In [13]:
def parse_pits(folder_path):
    '''
    Function to parse CAAML files in the specified folder
    '''

    files = [f for f in os.listdir(folder_path) if f.endswith(".xml")]  # List of all .xml files in the folder
    
    pits_list = []

    for file in files: # iterate through each file in the folder
        file_path = folder_path + "/" + file # create the file path
        pit = caaml_parser(file_path) # parse the file
        pits_list.append(pit)

    return pits_list

In [14]:
# Define folders and parse pits

pits_19_20 = parse_pits("snowpits/by_season/2019-2020")
pits_20_21 = parse_pits("snowpits/by_season/2020-2021")
pits_21_22 = parse_pits("snowpits/by_season/2021-2022")
pits_22_23 = parse_pits("snowpits/by_season/2022-2023")
pits_23_24 = parse_pits("snowpits/by_season/2023-2024")

all_pits = pits_19_20 + pits_20_21 + pits_21_22 + pits_22_23 + pits_23_24 # list of all pits


### Core Info

Table
- Number of pits
- Unique users
- Pits by Professionals
- Pits by non-professionals
- Unique Countries

In [15]:
core_info_list = []

for pit in all_pits:

    core_info_dict = {
        # metadata
        "PitID": pit.coreInfo.pitID,
        "Date": pit.coreInfo.date,
        # User
        "SnowPilot Username": pit.coreInfo.user.username,
        "Professional": pit.coreInfo.user.professional,
        "Operation Name": pit.coreInfo.user.operationName,
        # Location
        "Latitude": pit.coreInfo.location.latitude,
        "Longitude": pit.coreInfo.location.longitude,
        "Elevation": pit.coreInfo.location.elevation,
        "Aspect": pit.coreInfo.location.aspect,
        "Slope Angle": pit.coreInfo.location.slopeAngle,
        "Country": pit.coreInfo.location.country,
        "Region": pit.coreInfo.location.region,
        "Pit Near Avalanche": pit.coreInfo.location.pitNearAvalanche,
        "Pit Near Avalanche Location": pit.coreInfo.location.pitNearAvalancheLocation,
    }
    core_info_list.append(core_info_dict)

core_info_df = pd.DataFrame(core_info_list)

In [16]:
# Create a summary table of core_info_df
summary_table = {
    "Metric": [
        "Total Pits",
        "Date Range",
        "Unique Users",
        "Professional Users",
        "Non-Professional Users",
        "Unique Countries",
        "Pits Near Avalanche",
    ],
    "Value": [
        len(core_info_df),  # Total quantity of PitID
        f"{core_info_df['Date'].min()} to {core_info_df['Date'].max()}",  # Date Range
        core_info_df['SnowPilot Username'].nunique(),  # Unique Usernames
        core_info_df['Professional'].sum(),  # Count of Professional = True
        (~core_info_df['Professional']).sum(),  # Count of Professional = False
        core_info_df['Country'].nunique(),  # Unique Countries
        core_info_df['Pit Near Avalanche'].sum(),  # Count of Pit Near Avalanche = True
    ]
}

# Create DataFrame for the summary table
summary_df = pd.DataFrame(summary_table)

# Display the summary table
print("Summary of Core Info Data:")
print(summary_df)

# Count of each unique value for "Pit Near Avalanche Location"
print("\nPit Near Avalanche Locations:")
avalanche_locations = core_info_df[core_info_df['Pit Near Avalanche'] == True]['Pit Near Avalanche Location'].value_counts()
print(avalanche_locations)

Summary of Core Info Data:
                   Metric                     Value
0              Total Pits                     31170
1              Date Range  1998-12-17 to 2024-09-25
2            Unique Users                      3854
3      Professional Users                     19891
4  Non-Professional Users                     11279
5        Unique Countries                        31
6     Pits Near Avalanche                       945

Pit Near Avalanche Locations:
Pit Near Avalanche Location
crown    480
flank    240
other    146
Name: count, dtype: int64


### Snow Profile

In [17]:
snow_profile_list = []

for pit in all_pits:
    numPrimaryGrainForm = 0 # initialize
    numPrimaryGrainSize = 0 # initialize
    for layer in pit.snowProfile.layers: # iterate through each layer in the pit
        if layer.grainFormPrimary is not None: # if the layer has a primary grain form
            numPrimaryGrainForm += 1 # increment the number of primary grain forms
            if layer.grainFormPrimary.grainSizeAvg is not None: # if the layer has a primary grain size
                numPrimaryGrainSize += 1 # increment the number of primary grain sizes

    snow_profile_dict = {
        "PitID": pit.coreInfo.pitID,
        "Measurement Direction": pit.snowProfile.measurementDirection,
        "Profile Depth": pit.snowProfile.profileDepth,
        "HS": pit.snowProfile.hS,
        # Surface Conditions    
        "Foot Penetration": pit.snowProfile.surfCond.penetrationFoot,
        "Ski Penetration": pit.snowProfile.surfCond.penetrationSki,
        #Layers
        "Num Layers": len(pit.snowProfile.layers),
        "num Layers wPrimary Grain Form": numPrimaryGrainForm,
        "num Layers wPrimary Grain Size": numPrimaryGrainSize,
        # Temp Profile
        "Temp Profile": True if pit.snowProfile.tempProfile is not None else False,
        "Num Temp Obs": len(pit.snowProfile.tempProfile) if pit.snowProfile.tempProfile is not None else 0,
        # Density Profile
        "Density Profile": True if pit.snowProfile.densityProfile is not None else False,
        "Num Density Obs": len(pit.snowProfile.densityProfile) if pit.snowProfile.densityProfile is not None else 0,

    }
    snow_profile_list.append(snow_profile_dict)

snow_profile_df = pd.DataFrame(snow_profile_list)

### Stability Tests